In [1]:
"""
Created on Sun Mar  3 21:42:16 2019

@author: André Neves
"""
import sys
import gdal
from utils import visualization as viz
from utils import data
import numpy as np
from tqdm import tqdm
from sklearn.metrics import classification_report, confusion_matrix, cohen_kappa_score
import matplotlib
import matplotlib.pyplot as plt

# inicialize data location
DATA_FOLDER = "../sensing_data/"
ROI = "vila-de-rei/"

SRC_FOLDER = DATA_FOLDER + "results/" + ROI


SRC = SRC_FOLDER + "timeseries/"

FCG_SRC = SRC_FOLDER + "fgc/"

GT_SRC = SRC_FOLDER + "GT/"
COS_SRC = DATA_FOLDER + "clipped/" + ROI

OUT_RASTER = DATA_FOLDER + "results/" + ROI + \
    "timeseries/xgb/GT_group1_classification.tiff"
OUT_RASTER_2 = DATA_FOLDER + "results/" + ROI + \
    "timeseries/xgb/GT_group2_classification.tiff"

REF_FILE = DATA_FOLDER + "clipped/" + ROI + \
    "ignored/static/clipped_sentinel2_B08.vrt"

G1_CLASS = "../paper/EPIA2020/results/group1/boosted_20px_static_group1_classification.tiff"
G2_CLASS = "../paper/EPIA2020/results/group2/boosted_20px_static_group2_classification.tiff"

In [2]:
def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=True,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting normalize=True.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)

    # Only use the labels that appear in the data
    # classes = classes[unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    fig, ax = plt.subplots(figsize=(11, 10))
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    #ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    plt.savefig(title+'.pdf')
    return ax

In [3]:
result_10m_boosted_split = gdal.Open(G2_CLASS, gdal.GA_ReadOnly)

result_10m_boosted = gdal.Open(G1_CLASS, gdal.GA_ReadOnly)

# get result data
result_10m_boosted = result_10m_boosted.GetRasterBand(1).ReadAsArray()
result_10m_boosted_split = result_10m_boosted_split.GetRasterBand(1).ReadAsArray()

# open and map groung truth
cos = gdal.Open(COS_SRC + "clipped_cos_50982.tif", gdal.GA_ReadOnly)
cos = cos.GetRasterBand(1).ReadAsArray()
cos = cos[:result_10m_boosted_split.shape[0], :result_10m_boosted_split.shape[1]]

print("Mapping cos...")
cos_g1 = np.array([data._class_map(yi)
                for yi in tqdm(cos.flatten())])
cos_g2 = np.array([data._class_split_map(yi)
                    for yi in tqdm(cos.flatten())])

print("Mapping split...") 
result_10m_boosted_split_mapped = np.array([split_map(yi)
                for yi in tqdm(result_10m_boosted_split.flatten())])

  0%|▏                                                                                                                                                                                                                                    | 16969/19386625 [00:00<01:54, 169637.99it/s]

Mapping cos...


  0%|                                                                                                                                                                                                                                                     | 0/19386625 [00:00<?, ?it/s]

Mapping split...


NameError: name 'split_map' is not defined

In [ ]:
classes_cos = ["ESTRUTURAS", "NATURAL", "ÁGUA"]
 
print("cos vs split")
kappa = cohen_kappa_score(cos, result_10m_boosted_split_mapped)
print(f'Kappa: {kappa}')
print(classification_report(cos, result_10m_boosted_split_mapped))
print(confusion_matrix(cos, result_10m_boosted_split_mapped))

print("cos vs boosted")
kappa = cohen_kappa_score(cos, result_10m_boosted.flatten())
print(f'Kappa: {kappa}')
print(classification_report(cos, result_10m_boosted.flatten()))
print(confusion_matrix(cos, result_10m_boosted.flatten()))

print("boosted vs split")
kappa = cohen_kappa_score(result_10m_boosted.flatten(), result_10m_boosted_split_mapped)
print(f'Kappa: {kappa}')
print(classification_report(result_10m_boosted.flatten(), result_10m_boosted_split_mapped))
print(confusion_matrix(result_10m_boosted.flatten(), result_10m_boosted_split_mapped))